## `prolintpy`: Overview and Tutorial

prolintpy is a lightweight python library that aimes to automate the analysis and visualization of Protein-Lipid interactions.

It is districbuted as part of the ProLint framework with the aim of bridging the widening gap between data generation and gaining insight on biologically-relevant interactions between lipids and proteins. prolintpy is the library that the ProLint webserver uses on the backend to automate topology generation and analysis of lipid-protein interactions. Nevertheless, prolintpy includes a dedicated interface for the visualization of lipid-protein interactions similar to the webserver which can be accessed through  JupyterLab/Jupyter Notebook.

You can use prolintpy for the following:

1. Automatically generate a topology description of your system (no tpr file needed)
2. Calculate contact-based metrics for lipid-protein interactions
3. Calculate 2D and 3D densities (3D densities are work in progress)
4. Calculate physics-based properties (in progress)
5. Interactively visualize lipid-protein interactions


## Section 0: Installation 

In [ ]:
# Only Python v3.6 or v3.7 are supported. Python v3.8 may not work.

# Method 1
pip install prolintpy

# Method 2
git clone https://github.com/ProLint/prolintpy.git
cd prolintpy
conda env create -f environment.yml
conda activate prolint

# Method 3
git clone https://github.com/ProLint/prolintpy.git
cd prolintpy
python setup.py install

# Method 4 (if MDTraj causes problem)
conda create --name prolint python=3.7
conda activate prolint
conda install -c conda-forge mdtraj
python -m pip install prolintpy

`prolintpy` relies on `MDTraj` to read input data files, as such, it is the only module that has to be imported alongside `prolintpy`: 

In [1]:
import numpy as np
import mdtraj as md
import prolintpy as pl

## Section 1: Loading data and topology description

Load the data using MDTraj

#### Note
The system we are loading contains only lipids and protein beads (water and ions have been removed). <br>
This is important otherwise prolintpy may treat water as if it were a lipid.

The image below shows the system we are going to load and work with. The protein is shown with magenta, lipids are colored with blue (POPS), orange (POPE), and green (CHOL). 

<!-- ![title](images/sys_1_front.png)  -->
<img src="images/sys_1.png" alt="system_front" style="width:1200px;"/>

In [2]:
# MDTraj v1.9.6 seems to have a bug when reading Martini coordinate files. 
# That why prolintpy currently forces the installation of v1.9.5
t = md.load('./data/test_data_1.xtc', top='./data/test_data_1.gro')

In [3]:
t

<mdtraj.Trajectory with 17 frames, 23820 atoms, 3240 residues, and unitcells at 0x7fc0592b3350>

### Load the data to prolintpy and define the protein and lipid topology

We first specify the resolution of the input data and indicate if we want to combine the proteins (only applicable if there are more than one protein in the system). 
<br>Combining proteins will result in the calculated metrics being averages of all copies. In our system we only have one protein so we do not need it. 

In [4]:
resolution = "martini"
combine_proteins = False
lipids = pl.Lipids(t.topology, resolution=resolution)
proteins = pl.Proteins(t.topology, resolution=resolution).system_proteins(merge=combine_proteins)

## Extracting information from the input system

Get all the lipid residues in the system

In [5]:
lipids.lipid_names()

array(['POPE', 'POPS', 'CHOL'], dtype=object)

Get the names of the different lipids as well as their count

In [6]:
lipids.lipid_count()

{'POPE': 652, 'POPS': 652, 'CHOL': 652}

Get a pandas DataFrame for the defined systems

In [7]:
lipids.ldf.head()

,serial,name,element,resSeq,resName,chainID,segmentID
2956,2957,NH3,N,1285,POPE,0,
2957,2958,PO4,P,1285,POPE,0,
2958,2959,GL1,VS,1285,POPE,0,
2959,2960,GL2,VS,1285,POPE,0,
2960,2961,C1A,C,1285,POPE,0,


Retrieve the residue IDs of all cholesterol lipids

In [8]:
lipids.ldf[lipids.ldf.resName == "CHOL"].resSeq.unique()

array([1937, 1938, 1939, 1940, 1941, 1942, 1943, 1944, 1945, 1946, 1947,
       1948, 1949, 1950, 1951, 1952, 1953, 1954, 1955, 1956, 1957, 1958,
       1959, 1960, 1961, 1962, 1963, 1964, 1965, 1966, 1967, 1968, 1969,
       1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980,
       1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991,
       1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002,
       2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013,
       2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024,
       2025, 2026, 2027, 2028, 2029, 2030, 2031, 2032, 2033, 2034, 2035,
       2036, 2037, 2038, 2039, 2040, 2041, 2042, 2043, 2044, 2045, 2046,
       2047, 2048, 2049, 2050, 2051, 2052, 2053, 2054, 2055, 2056, 2057,
       2058, 2059, 2060, 2061, 2062, 2063, 2064, 2065, 2066, 2067, 2068,
       2069, 2070, 2071, 2072, 2073, 2074, 2075, 2076, 2077, 2078, 2079,
       2080, 2081, 2082, 2083, 2084, 2085, 2086, 20

List the proteins found in the system and store the first one (the only one here) in a variable. 
`prolintpy` derives topology information for proteins from the input coordinate file. 
<br>Two proteins will be considered the same if they are entirely identical (same number of residues, completely identical order and type of atoms/beads). 

In [9]:
proteins

[<prolintpy.Protein containing 1 replicate(s) of Protein0 and 1284 beads each>]

In [10]:
protein = proteins[0]

Get various protein information. Note that to get a dataframe 

In [11]:
protein.name = "MyProtein" # Give the protein a better name

In [12]:
protein.n_residues

1284

In [13]:
print (protein.first_residue, protein.last_residue)

1 1284


In [14]:
protein.counter

1

Get the indices for residues 50, 60, and 70

In [15]:
protein.get_indices([50, 60, 70])

Using the available dataframe


[array([124, 125]), array([155, 156, 157, 158, 159]), array([179, 180])]

### Why `prolintpy` is easy to scale-up 

If the input system contains only one copy of only one protein type (as in this example) then `proteins` will be a list of only one element. This entails a little bit extra work to get the protein out of the list, but provides much more flexibility in handling more complex system setups. You can use the `counter` option alongside the length of the `proteins` list to extract information about proteins in the system dynamically.

For instance, to get a DataFrame representation for each protein in the system dynamically (that is without knowing anything about the composition of the input system), we can do that very easily<br>
One way of doing that is the following syntax: 

In [16]:
def get_dataframes(proteins):
    """
    Takes as input a prolintpy.Protein object and returns a list of DataFrame elements 
    for each copy of each protein in the system. 
    """
    dataframe_list = [protein.dataframe[protein_copy] for protein in proteins for protein_copy in range(protein.counter)]
    return dataframe_list

In [17]:
# returns a list of DataFrame elements 
get_dataframes(proteins)

[      serial name element  resSeq resName  chainID segmentID
 0          1   BB       B       1     ARG        0          
 1          2  SC1       S       1     ARG        0          
 2          3  SC2       S       1     ARG        0          
 3          4   BB       B       2     GLN        0          
 4          5  SC1       S       2     GLN        0          
 5          6   BB       B       3     ARG        0          
 6          7  SC1       S       3     ARG        0          
 7          8  SC2       S       3     ARG        0          
 8          9   BB       B       4     TYR        0          
 9         10  SC1       S       4     TYR        0          
 10        11  SC2       S       4     TYR        0          
 11        12  SC3       S       4     TYR        0          
 12        13   BB       B       5     MET        0          
 13        14  SC1       S       5     MET        0          
 14        15   BB       B       6     GLU        0          
 15     

## Section 2: Contact-based metrics

Redefine topologies, but now we'll only select CHOL and POPS lipids

In [18]:
resolution = "martini"
combine_proteins = False
lipids = pl.Lipids(t.topology, resolution=resolution, lipid_names=["CHOL", "POPS"])
proteins = pl.Proteins(t.topology, resolution=resolution).system_proteins(merge=combine_proteins)

### Build a prolintpy.ComputeContacts object

We use this object for all contact calculations

In [19]:
contacts = pl.ComputeContacts(t, proteins, lipids)

In [20]:
contacts

Given a list of residues and a cutoff distance, calculate all neighboring lipids: 

In [21]:
residues = [*range(85, 130)]
cutoff = 0.5 # nm

In [22]:
# We will calculate contacts with the list of residues defined above and store them in the result variable. 
# This variable will have all of the contact information we require. 
results = contacts.compute_neighbors(t, residues=residues, cutoff=cutoff, grouped=False)

# The command below will calcualte contacts with all residues, but it may take a while longer to finish. 
# results = contacts.compute_neighbors(t, cutoff=cutoff, grouped=False)

Working on protein copy: 0


`results` is a nested dictionary, encoding the following information hierarchy: <br>
protein name > protein copy > residue ID > prolintpy.LPContacts object

In [23]:
results

{'Protein0': {0: {85: <prolintpy.LPContacts for residue 85>,
   86: <prolintpy.LPContacts for residue 86>,
   87: <prolintpy.LPContacts for residue 87>,
   88: <prolintpy.LPContacts for residue 88>,
   89: <prolintpy.LPContacts for residue 89>,
   90: <prolintpy.LPContacts for residue 90>,
   91: <prolintpy.LPContacts for residue 91>,
   92: <prolintpy.LPContacts for residue 92>,
   93: <prolintpy.LPContacts for residue 93>,
   94: <prolintpy.LPContacts for residue 94>,
   95: <prolintpy.LPContacts for residue 95>,
   96: <prolintpy.LPContacts for residue 96>,
   97: <prolintpy.LPContacts for residue 97>,
   98: <prolintpy.LPContacts for residue 98>,
   99: <prolintpy.LPContacts for residue 99>,
   100: <prolintpy.LPContacts for residue 100>,
   101: <prolintpy.LPContacts for residue 101>,
   102: <prolintpy.LPContacts for residue 102>,
   103: <prolintpy.LPContacts for residue 103>,
   104: <prolintpy.LPContacts for residue 104>,
   105: <prolintpy.LPContacts for residue 105>,
   106:

Extract contact data for a particular residue

In [24]:
# Proteins are named ProteinN, where N is 0, 1, 2, ..., number of copies in the system. 
# In our system we only have one copy of one protein, so the default name given is: Protein0
results['Protein0'][0][85]

<prolintpy.LPContacts for residue 85>

In [25]:
contact_r88 = results['Protein0'][0][88]

In [26]:
contact_r88

<prolintpy.LPContacts for residue 88>

Not all interactions are equal! Residue 88 interacts with only one cholesterol lipid during the length of the input trajectory. However, it forms several interactions with POPS lipids, even though the input test trajectory is quite short. <br>The output is a dictionary with lipids as keys and the contact duration as dictionary values. The time units here are the same as MDTraj. 

In [27]:
contact_r88.contacts

{'POPS': [300000.0, 900000.0, 300000.0, 600000.0, 1200000.0, 300000.0],
 'CHOL': [3900000.0]}

We can also retrieve the residue id of the lipids that form the interactions above. <br>
This is very useful if we want to build custom metrics or just in general customize the workflow

In [28]:
contact_r88.lipids

{'POPS': array([1677, 1773, 1817, 1888, 1889, 1934]), 'CHOL': array([2951])}

Occupancy is a binary measure so we need to retrieve it separately

In [29]:
contact_r88.occupancy

{'POPS': array([1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.]),
 'CHOL': array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])}

### Customizability 

Altogether, the `prolintpy` capabilities highlighted above make it clear how easy it is to customize the workflow to your use-case. This is because analysis is not a closed system, and `prolintpy` provides access 
to its internal data at every step of the way. You can use the `result` dictionary above and loop through the different options without worrying about any of the other features of `prolintpy`. 

Nevertheless, `prolintpy` provides several helper functions and features which make working with the contact informations above really easy

### Helper functions 

Helper functions allow you to efficiently get data from the `results` dictionary defined above. 

For instance, retrieve contact information for a residue with all lipids in the system

In [30]:
pl.retrieve_contacts(results, 88)

defaultdict(list,
            {'POPS': [array([ 300000.,  900000.,  300000.,  600000., 1200000.,  300000.])],
             'CHOL': [array([3900000.])]})

In [31]:
# If we want to retrieve occupancy values we can indicate that: 
pl.retrieve_contacts(results, 88, contacts='occupancy')

defaultdict(list,
            {'POPS': [array([1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1.])],
             'CHOL': [array([1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0.])]})

Retrieve contact information for a specific residue-lipid pair

In [34]:
pl.retrieve_contacts_flat(results, 88, lipid="CHOL")

array([3900000.])

In [36]:
pl.retrieve_contacts_flat(results, 88, lipid='POPS')

array([ 300000.,  900000.,  300000.,  600000., 1200000.,  300000.])

### Build a pandas DataFrame

pandas DataFrames are the most convenient way to manipulate contact information. <br>
Building a DataFrame using `prolintpy` is a straightforward process. 

In [37]:
df = pl.contacts_dataframe(results, proteins, t, radius=cutoff)

In [38]:
df.head()

,Mean_Duration,Longest_Duration,Sum_of_all_Contacts,Lipid_Number,Normalized_Lipid_Number,Occupancy,Protein,Lipids,Radius,ResID,ResName
0,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,85,ARG
1,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,86,GLN
2,0.3,0.3,1.2,0.2500,0.2500,23.529412,Protein0,CHOL,0.5,87,ARG
3,3.9,3.9,3.9,0.8125,0.8125,76.470588,Protein0,CHOL,0.5,88,TYR
4,0.0,0.0,0.0,0.0000,0.0000,0.000000,Protein0,CHOL,0.5,89,MET


Sort by the most interacting residues in terms of the Longest_Duration metric

In [39]:
df.sort_values('Longest_Duration', ascending=False).head()

,Mean_Duration,Longest_Duration,Sum_of_all_Contacts,Lipid_Number,Normalized_Lipid_Number,Occupancy,Protein,Lipids,Radius,ResID,ResName
6,1.70,4.5,5.1,1.0625,1.0625,88.235294,Protein0,CHOL,0.5,91,LYS
3,3.90,3.9,3.9,0.8125,0.8125,76.470588,Protein0,CHOL,0.5,88,TYR
43,1.95,3.6,3.9,0.8125,0.8125,70.588235,Protein0,CHOL,0.5,128,PHE
36,1.50,3.0,4.5,0.9375,0.9375,64.705882,Protein0,CHOL,0.5,121,TRP
40,1.95,3.0,3.9,0.8125,0.8125,58.823529,Protein0,CHOL,0.5,125,LEU


Get the residue ids and indices of the top 5 residues interacting with cholesterol as measured by the Longest_Duration metric

In [40]:
top_residues = df[df.Lipids == "CHOL"].sort_values('Longest_Duration', ascending=False).ResID.to_list()[:10]
print ("Most interacting residues are: ", top_residues, end="\n" + '~' * 80)

Most interacting residues are:  [91, 88, 128, 121, 125, 124, 117, 118, 122, 114]
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [41]:
[f'Residue {res} with indices: {proteins[0].get_indices([res], suppress=True)[0]}' for res in top_residues]

['Residue 91 with indices: [225 226 227 228]',
 'Residue 88 with indices: [220 221]',
 'Residue 128 with indices: [304 305]',
 'Residue 121 with indices: [292 293]',
 'Residue 125 with indices: [299 300]',
 'Residue 124 with indices: [297 298]',
 'Residue 117 with indices: [284 285]',
 'Residue 118 with indices: [286 287]',
 'Residue 122 with indices: [294 295]',
 'Residue 114 with indices: [279 280]']

## Section 3: Visualization 

In [42]:
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## Scatter Plot

Once you have build a pandas DataFrame from the calculated contacts (by, for instance, running the contacts_dataframe() function), you can provide it as input to the appropriate prolintpy visualization apps:

In [43]:
pl.show_points(df, size=10)

Loading BokehJS ...

## Distance Calculations & Visualization

A very important (perhaps the most important?) calculation that is commonly done in lipid-protein interaction studies is measuring the distance between a residue and a lipid as a function of simulation time. This gives you a clear idea if the lipid is interacting preferentially with a residue or not. prolintpy provides two different ways to get distance information on lipid-protein interactions. The first method, presented in this section, is automated and relies on the prior calculation of contact-based metrics.

The way it works is that it goes through the calculated metrics, sorts them, and gets the top-ranking residues and lipids. It then goes over each residue and lipid combination and gets the best/strongest contact (that is, the contact that is maintained most strongly between the specific residue and lipid).

In [44]:
dist = contacts.compute_distances(t, proteins[0], [*range(85, 91)], 'POPS', 'PO4', percentile_co=0.05, distance_co=0.7)

In [45]:
pl.show_distances(dist)

Loading BokehJS ...

The second way `prolintpy` calculates and visualizes distances is by not relying on any prior calcualted metrics. Instead, you simply supply the protein
and list of residues along with threshold arguments, and `prolintpy` will then calculate distance measurements.

Given a list of input residues,
this function will loop through all the lipids in the system and display distances with best ranking lipids. Ranking is decided
based on the following parameters:

| Argument      | Default | Description                                                                                                              |
| ------------- | ------- | ------------------------------------------------------------------------------------------------------------------------ |
| distance_co   | 0.7     | A cutoff distance (nm) that a lipid must satisfy for `percentile_co` frames of the trajectory.                           |
| percentile_co | 0.05    | The percentage of the trajectory (measured in frames) that a lipid must be within the `distance_co` for it to be stored. |

Here is an example application:

In [46]:
from prolintpy.core.computecontacts import retrieve_distances

We first build a dictionary of lipids and residues that are interacting preferably and then we compute the distances between them. <br>
You can use the `top_nr` argument to indicate how many top ranking results you want to consider. 

This application is more computationally expensive since it requires the prior calculation of contact-based metrics, but it is much better in extracting good contacts. <br>
It also does not require any threshold definitions or manual input of lipid and residue ids. 

In [47]:
distances_dict, SYSTEM_LIPIDS, lipids_found = retrieve_distances(df, group_lipids=False, resolution=resolution, lipids=lipids, top_nr=30)
distances = contacts.compute_lipid_distances(t, proteins[0], distances_dict, SYSTEM_LIPIDS, lipids_found)

In [48]:
pl.show_metric_distances(distances)

Loading BokehJS ...

## Contact Projection

May need to execute: ```jupyter-nbextension enable nglview --py --sys-prefix```
if visualization does not show

In [49]:
# if you have multiple cutoffs then you also need to filter the dataframe using one of the cutoffs. 
contact_values = df[df.Lipids == "CHOL"].Longest_Duration.to_list()

# residue_list is only required when working with a subset of protein residues. 
residues = df[df.Lipids == "CHOL"].ResID.to_list()

pl.show_contact_projection(t, bf=contact_values, protein=proteins[0], residue_list=residues, ngl_repr='spacefill', cmap="Reds")

NGLWidget()

# What else? 

`prolintpy` supports other application which are not shown here. They are part of the `prolintpy.vis` module. 
`prolintpy` is part of ProLint, a framework to automate analysis and visualization of lipid-protein interactions. <br>

#### Source Code & Documentation
The source code is available here: https://github.com/ProLint/prolintpy <br>
The documentation is available here: https://prolint.github.io/prolintpy <br>
The webserver and its documentation: https://prolint.ca <br>
The source code of the webserver: https://github.com/ProLint/ProLint <br>

 

## Exercise

Test the above commands using a system that contains multiple proteins in different number of copies/replicates. <br>
The test files `test_data_2.xtc`  and `test_data_2.gro` contain a system that has four copies/replicates of one protein type, with 60+ different lipid types (see image below). 

<!-- ![title](images/sys_1_front.png)  -->
<img src="images/sys_2.png" alt="system_front" style="width:1200px;"/>

In [ ]:
t = md.load('./data/test_data_2.xtc', top='./data/test_data_2.gro')